In [1]:
import sqlite3
import pandas as pd
from datetime import datetime
import random

In [2]:
database = 'database.sqlite'
conn = sqlite3.connect(database)

In [3]:
player_data = pd.read_sql("SELECT * FROM Player;", conn)
#print(player_data.columns.values)
#leagues = pd.read_sql("Select * from League;",conn)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", conn)


match_data = pd.read_sql("SELECT * FROM Match;", conn)
#match_data.to_csv('match_data.csv',encoding='utf-8')
#print(match_data.columns.values)
#print(match_data)
teams = pd.read_sql("SELECT * FROM Team;", conn)
teams.to_csv('teams.csv',encoding='utf-8')

Team_Stats=pd.read_sql("SELECT * FROM Team_Attributes",conn)


In [4]:

result_entries = []
feature_entries = []
count =0

for i in range(len(match_data)):
    record = match_data.loc[i, :]
    home_goals = int(record['home_team_goal'])
    away_goals = int(record['away_team_goal'])
    if home_goals > away_goals:
        result = 'HOME'
    elif away_goals > home_goals:
        result = 'AWAY'
    else:
        result = 'DRAW'
    home_team = teams[teams.team_api_id == record['home_team_api_id']]['team_long_name'].values[0]
    away_team = teams[teams.team_api_id == record['away_team_api_id']]['team_long_name'].values[0]
    home_player_X1=record['home_player_X1']
    home_player_X2=record['home_player_X2']
    home_player_X3=record['home_player_X3']
    home_player_X4=record['home_player_X4']
    home_player_X5=record['home_player_X5']
    away_player_X1=record['away_player_X1']
    away_player_X2=record['away_player_X2']
    away_player_X3=record['away_player_X3']
    away_player_X4=record['away_player_X4']
    away_player_X5=record['away_player_X5']
    home_player_Y1=record['home_player_Y1']
    home_player_Y2=record['home_player_Y2']
    home_player_Y3=record['home_player_Y3']
    home_player_Y4=record['home_player_Y4']
    home_player_Y5=record['home_player_Y5']
    away_player_Y1=record['away_player_Y1']
    away_player_Y2=record['away_player_Y2']
    away_player_Y3=record['away_player_Y3']
    away_player_Y4=record['away_player_Y4']
    away_player_Y5=record['away_player_Y5']
    
      
    
    result_entry = {'home_team_api_id':record['home_team_api_id'],'away_team_api_id':record['away_team_api_id']
,'home_team_goal': record['home_team_goal'], 'away_team_goal': record['away_team_goal'],
                    'result': result, 'home_player_X1':home_player_X1,'home_player_X2':home_player_X2,'home_player_X3':home_player_X3,'home_player_X4':home_player_X4,'home_player_X5':home_player_X5,
                   'home_player_Y1':home_player_Y1,'home_player_Y2':home_player_Y2,'home_player_Y3':home_player_Y3,'home_player_Y4':home_player_Y4,'home_player_Y5':home_player_Y5,
                   'away_player_X1':away_player_X1,'away_player_X2':away_player_X2,'away_player_X3':away_player_X3,
                    'away_player_X4':away_player_X4,'away_player_X5':away_player_X5,'away_player_Y1':away_player_Y1,
                    'away_player_Y2':away_player_Y2,'away_player_Y3':away_player_Y3,'away_player_Y4':away_player_Y4,
                    'away_player_Y5':away_player_Y5
                  }
    
    
    result_entries.append(result_entry)
result_df = pd.DataFrame(result_entries)
result_df['result'].replace('HOME',int('+1'),inplace=True)
result_df['result'].replace('AWAY',int('-1'),inplace=True)
result_df['result'].replace('DRAW',int('0'),inplace=True)


In [5]:
#Getting Goals Data from Previous 5 game 
#Getting Goals Data from Previous 5 game 
additional_entries = []
def generate_features(home_team, away_team,result_df, games=5):
  
    total_home_games = result_df[(result_df.home_team_api_id == home_team)].tail(games) 
    total_home_goals_scored = sum(total_home_games['home_team_goal'])
    total_home_goals_conceded = sum(total_home_games['away_team_goal'])
    
    total_away_games = result_df[(result_df.away_team_api_id == away_team)].tail(games)
    total_away_goals_scored = sum(total_away_games['away_team_goal'])
    total_away_goals_conceded = sum(total_away_games['home_team_goal'])
    
    return total_home_goals_scored,total_home_goals_conceded,total_away_goals_scored,total_away_goals_conceded
    


for i in range(len(match_data)):
    record = match_data.loc[i, :]
    test1=record['home_team_api_id']
    #print(record['away_team_api_id'])
    test2=record['away_team_api_id']
    
    a,b,c,d = generate_features(test1,test2,result_df)
    add_features={'home_total_scored':a,'home_total_conceded':b,'away_total_scored':c,'away_total_conceded':d}
    additional_entries.append(add_features)
additional_df = pd.DataFrame(additional_entries)

#print(additional_df)


In [6]:
result_df['home_total_scored']= additional_df['home_total_scored']
result_df['home_total_conceded']= additional_df['home_total_conceded']
result_df['away_total_scored']= additional_df['away_total_scored']
result_df['away_total_conceded']= additional_df['away_total_conceded']

In [56]:
import numpy as np
from sklearn import preprocessing, cross_validation, neighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier

result_df.dropna(inplace = True)
labels = result_df.loc[:,'result']
features = result_df.drop('result', axis = 1)

clf = neighbors.KNeighborsClassifier(n_neighbors=6000)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2)

print(X_train.shape)
print(y_train.shape)

RF_clf = RandomForestClassifier(n_estimators = 2)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2)

GNB_clf = GaussianNB()
Tree_Clf=tree.DecisionTreeClassifier()

clf_Ada = AdaBoostClassifier(n_estimators=100)
clf.fit(X_train, y_train)
RF_clf.fit(X_train, y_train)
GNB_clf.fit(X_train, y_train)
clf_Ada.fit(X_train, y_train)



#GNB_clf.fit(X_train,y_train)
KNNaccuracy = clf.score(X_test,y_test)
RFaccuracy = RF_clf.score(X_test,y_test)
GNBaccuracy = GNB_clf.score(X_test,y_test)
Adaaccuracy = clf_Ada.score(X_test,y_test)

print ("accuracy with KNN "+str(KNNaccuracy))
print ("accuracy with RF "+str(RFaccuracy))
print ("accuracy with GaussianNB "+str(GNBaccuracy))
print ("accuracy with AdaBoostClassifier"+str(Adaaccuracy))


(19317, 28)
(19317,)
accuracy with KNN 0.455279503106
accuracy with RF 0.923602484472
accuracy with GaussianNB 0.796273291925
accuracy with AdaBoostClassifier0.717598343685


                     away_team       date  home_home_goals_conceded  \
0                 Beerschot AC 2008-08-17                         0   
1             Sporting Lokeren 2008-08-16                         0   
2               RSC Anderlecht 2008-08-16                         0   
3                    RAEC Mons 2008-08-17                         0   
4            Standard de Liège 2008-08-16                         0   
5               Club Brugge KV 2008-09-24                         2   
6                  KV Kortrijk 2008-08-16                         0   
7         Royal Excel Mouscron 2008-08-16                         0   
8           Sporting Charleroi 2008-08-16                         0   
9                  KV Kortrijk 2008-11-01                         2   
10          Sporting Charleroi 2008-10-31                         2   
11                    KAA Gent 2008-11-02                         3   
12                   RAEC Mons 2008-11-01                         3   
13    